<h1>Numpy and Pandas Home Assignment</h1>
In this assignment, we will use pandas and numpy to do some fairly basic analysis on equities.

<h3>Imports</h3>

In [2]:
import datetime
import numpy as np
import pandas as pd
%matplotlib inline
import quandl # I used quandl because yahoo actions were depreciated
pd.core.common.is_list_like = pd.api.types.is_list_like
import pandas_datareader.data as web
# import fix_yahoo_finance as yf
quandl.ApiConfig.api_key = '4aMNMqjBPzy7zzuaKFnB'

<h3>Get data on the following four stocks from yahoo finance</h3>
<li>MS, GS, JPM, C
<li>Use the date range 2013/1/1/ to 2018/9/24

In [4]:
all_data = quandl.get_table('WIKI/PRICES', ticker = ['MS'], 
                        date = { 'gte': '2013-01-01', 'lte': '2018-09-24' }, 
                        paginate=True)

all_data.head()

,ticker,date,open,high,low,close,volume,ex-dividend,split_ratio,adj_open,adj_high,adj_low,adj_close,adj_volume
None,,,,,,,,,,,,,,
0,MS,2018-03-27,54.53,54.90,52.63,53.15,12209206.0,0.0,1.0,54.53,54.90,52.63,53.15,12209206.0
1,MS,2018-03-26,53.33,54.43,52.64,54.30,11824606.0,0.0,1.0,53.33,54.43,52.64,54.30,11824606.0
2,MS,2018-03-23,54.91,54.98,51.99,52.07,16396181.0,0.0,1.0,54.91,54.98,51.99,52.07,16396181.0
3,MS,2018-03-22,56.31,56.67,54.51,54.64,14593391.0,0.0,1.0,56.31,56.67,54.51,54.64,14593391.0
4,MS,2018-03-21,57.33,58.17,56.89,57.11,8417244.0,0.0,1.0,57.33,58.17,56.89,57.11,8417244.0


<h3>Compute the cross correlation coefficients on the stocks</h3>
<li>Create a df 'rets' with the one day percentage changes
<li>Calculate cross correlations on the rets df

In [3]:
# Isolate the `Adj Close` values and transform the DataFrame
daily_close_px = all_data.pivot(index='date', columns='ticker', values='adj_close')

# Calculate the daily percentage change for `daily_close_px`
rets = daily_close_px.pct_change()

# correlate
rets.corr(method='pearson', min_periods=1)

ticker,C,GS,JPM,MS
ticker,,,,
C,1.000000,0.795786,0.854757,0.813932
GS,0.795786,1.000000,0.814197,0.842153
JPM,0.854757,0.814197,1.000000,0.798226
MS,0.813932,0.842153,0.798226,1.000000


<h3>Select a pair of stocks to trade</h3>
<li>Choose the pair with the highest correlation
<li>You can just eyeball the result and pick the highest correlation pairs
<li>Create a new df 'pairs' with the returns columns of the highest correlation pairs

In [4]:
# it appears that GS and JPM are more correlated, but I'll use what fits the words below
pairs = rets[['C', 'JPM']].iloc[1:-1]

print(pairs.head())
# print(pairs['C'])

ticker             C       JPM
date                          
2013-01-03  0.003394 -0.002015
2013-01-04  0.025122  0.017725
2013-01-07  0.000947  0.001102
2013-01-08 -0.000235  0.001982
2013-01-09 -0.009892 -0.000659


<h3>Calculate the mean and std dev of the columns (returns of the two stocks)</h3>
<li>Note that line 1 contains a NaN. You'll have to get rid of this line
<li>df.describe() returns summary stats for all cols in a dataframe. You can use this to extract means and standard deviations

In [5]:
new_df = rets[['C','JPM']].iloc[1:-1]
c_std = new_df.describe()['C']['std']
j_std = new_df.describe()['JPM']['std']
c_mean = new_df.describe()['C']['mean']
j_mean = new_df.describe()['JPM']['mean']

print(c_std, j_std, c_mean, j_mean)

0.0150407028140531 0.012887162087069495 0.0005347629278196414 0.000866389513644403


<h3>Create a new column "Long_JPM_Short_C"</h3>
<li>If the return of C is greater than its mean + 0.25* std
<li> AND
<li>If the return of JPM is less than its mean - 0.25*std
<li>The value in this col should be True or False

In [6]:
new_df['Long_JPM_Short_C'] = np.where((new_df['C']>c_mean+0.25*c_std) & (new_df['JPM']<j_mean-0.25*j_std),True,False)

print(new_df.head())

ticker             C       JPM  Long_JPM_Short_C
date                                            
2013-01-03  0.003394 -0.002015             False
2013-01-04  0.025122  0.017725             False
2013-01-07  0.000947  0.001102             False
2013-01-08 -0.000235  0.001982             False
2013-01-09 -0.009892 -0.000659             False


<h3>Create a new column "Long_C_Short_JPM"</h3>
<li>If the return of JPM is greater than its mean + 0.25* std
<li> AND
<li>If the return of C is less than its mean - 0.25*std
<li>The value in this col should be True or False

In [7]:
new_df['Long_C_Short_JPM'] = np.where((new_df['JPM']>j_mean+0.25*j_std) & (new_df['C']<c_mean-0.25*c_std),True,False)

print(new_df)

ticker             C       JPM  Long_JPM_Short_C  Long_C_Short_JPM
date                                                              
2013-01-03  0.003394 -0.002015             False             False
2013-01-04  0.025122  0.017725             False             False
2013-01-07  0.000947  0.001102             False             False
2013-01-08 -0.000235  0.001982             False             False
2013-01-09 -0.009892 -0.000659             False             False
2013-01-10  0.018792  0.014955             False             False
2013-01-11 -0.011441 -0.000217             False             False
2013-01-14 -0.002834 -0.005635             False             False
2013-01-15  0.008290  0.010244             False             False
2013-01-16 -0.002114  0.010140             False             False
2013-01-17 -0.029190 -0.008116             False             False
2013-01-18  0.010184  0.000431             False             False
2013-01-22  0.002880  0.001722             False             F

<h3>Create a new column "Trade_return"</h3>
<li>If Long_JPM_Short_C is True and Long_C_Short_JPM is False, the value of this column should be the return on JPM minus the return on C
<li>If Long_JPM_Short_C is False and Long_C_Short_JPM is True, the value of this column should be the return on C minus the return on JPM

<li>Otherwise the value should be zero

In [8]:
# JminC = new_df['Long_JPM_Short_C'] ^ new_df['Long_C_Short_JPM']
# CminJ = new_df['Long_C_Short_JPM'] ^ new_df['Long_JPM_Short_C']

pairs['Trade_return'] = np.where(
    new_df['Long_JPM_Short_C'] & ~new_df['Long_C_Short_JPM'],
    new_df['JPM'] - new_df['C'],
    np.where(~new_df['Long_JPM_Short_C'] & new_df['Long_C_Short_JPM'],
    new_df['C'] - new_df['JPM'], 0))

total = pairs['Trade_return'].sum()
print(total)
#     np.where(new_df['Long_C_Short_JPM']==True & new_df['Long_JPM_Short_C']!=True,
#     CminJ, 0))
# Total = df['MyColumn'].sum()
# print (Total)
# 319

-0.5929190758688165


<h3>Calculate the sum of this new column</h3>
<li>Are you going to get rich?

In [9]:
print('no')

no
